In [ ]:
# LAst amended: 26th March, 2025
# RAG with csv files
# USes ollama models, ollama embeddings and chroma vector store

### Imports & Environment Variables 

In [23]:
# 1.0
import nest_asyncio
nest_asyncio.apply()

In [24]:
# 1.0 SimpleDirectoryReader can also load metadata from a dictionary
#     https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.readers.file import PagedCSVReader

# 1.1 The Settings is a bundle of commonly used resources used 
#     during the indexing and querying stage in a LlamaIndex workflow/application.
from llama_index.core import Settings

# 1.2 If using LocalAI
# https://docs.llamaindex.ai/en/stable/examples/llm/localai/
#from llama_index.llms.openai_like import OpenAILike

# 1.3 Ollama related
# https://docs.llamaindex.ai/en/stable/examples/embeddings/ollama_embedding/
from llama_index.embeddings.ollama import OllamaEmbedding
#from llama_index.llms.ollama import Ollama


# 1.4 Vector store related
import chromadb
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore

# 1.5 Misc
import os
import pandas as pd

In [25]:
# 2.0 Define embedding function

# embed_model = HuggingFaceEmbedding(
#                                     model_name="BAAI/bge-base-en-v1.5"
#                                    )

embed_model= OllamaEmbedding(
                                    model_name="nomic-embed-text",      # Using foundational model may be overkill
                                    base_url="http://localhost:11434",
                                    #dimensions=512,
                                    #ollama_additional_kwargs={"mirostat": 0},
                                  )

Settings.embed_model = embed_model

In [26]:
# 2.1 Settings can set the global configuration. Local configurations (transformations, LLMs, embedding models).
#     These can be passed directly into the interfaces that make use of them.

## Very very fast and Excellent
# pip install llama-index-llms-mistralai
from llama_index.llms.mistralai import MistralAI
llm = MistralAI(api_key="apikey")
Settings.llm = llm

### Load and Process CSV Data as Document

In [27]:
# 3.0 Reading data in pandas
#     It has nothing to do with subsequent analysis/usage

file_path = ('/home/ashok/Documents/csvrag/data/data.csv') # insert the path of the csv file
#file_path = ('/home/ashok/Downloads/csvrag/data/data.csv') # insert the path of the csv file
df = pd.read_csv(file_path)

,Roll_Number,fullName,Gender,BirthDate,age,MBA CGPA,garduation college,University,Degree,Specialization,percentage marks obtained during graduation,year when graduation was completed,name of the school attended for class 12,education board for class 12,academic stream chosen in class 12,percentage marks scored in class 12
0,1,Amit Sharma,Male,5/14/2002,23,6.69,Vivekananda Institute Of Professional Studies,Guru Gobind Singh Indraprastha University,B.C.A.,Computer Applications,53.83,2021,Holy Convent School,CBSE,Commerce,79.33
1,2,Pooja Verma,Female,8/25/1998,27,7.53,Asutosh College,Rajiv Gandhi Proudyogiki Vishwavidyalaya,B.E.,Biology,68.35,2020,Kishinchand Chellaram College,CBSE,Science,92.60
2,3,Rahul Kumar,Female,3/26/2002,23,7.70,St. Xavier'S College,Amity University,B.Com. (Hons.),Finance,74.16,2022,Delhi Public School,CBSE,Commerce,85.33
3,4,Neha Agarwal,Male,8/15/1997,26,5.82,School Of Commerce,Jaypee Institute Of Information Technology,B.Tech,Electronics And Communication Engineering,68.00,2019,G.A.V. Public School,CBSE,Science (Non-Medical),80.33
4,5,Rohan Das,Male,8/16/1999,25,6.80,Government Engineering College,Vellore Institute Of Technology,B.A. (Hons.),Economics,64.24,2021,Summer Fields School,CBSE,Science (Medical),67.60


### One way
Directly read csv file

In [28]:
# 4.0 PagedCSVReader displays each row in an LLM-friendly format. Each row as a separate document.
csv_reader = PagedCSVReader()

# 4.1
reader = SimpleDirectoryReader( 
                                input_files=[file_path],
                                file_extractor= {".csv": csv_reader}
                               )

# 4.2
docs = reader.load_data()

### Vector Store

In [31]:
# 5.0 Create client and a new collection
#     The following is  in-memory database and NOT a persistent collection.
#     chroma_client = chromadb.EphemeralClient()

# 5.1 This creates persistent collection. A folder by name of chromadb
#     is created and below that a chroma.sqlite3 database exists:

chroma_client = chromadb.PersistentClient(path="/home/ashok/Documents/chroma_db")

In [32]:
# 5.2 Check if collection exists. If so delete it.
#     Collections are the grouping mechanism for embeddings, documents, and metadata.
#     Chromadb can have multiple collections

if 'datastore' in chroma_client.list_collections():
    chroma_client.delete_collection("datastore")
    chroma_collection = chroma_client.create_collection("datastore")  
else:
    # Create collection afresh
    chroma_collection = chroma_client.create_collection("datastore")   

# 5.3 Get collection information:
chroma_collection    

Collection(name=datastore)

In [34]:
# 6.0 Set up a blank ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# 6.1
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [35]:
# 6.2 Takes docs and storage context:
#     Repeating this operation, doubles the number of vectors/records in the collection

index = VectorStoreIndex.from_documents(
                                         docs,
                                         storage_context=storage_context,
                                         show_progress= False                 # Show progress bar
                                        )

Generating embeddings: 100%|██████████████████████████████████████████████████████████| 500/500 [00:06<00:00, 76.84it/s]


CPU times: user 1.08 s, sys: 229 ms, total: 1.31 s
Wall time: 7.99 s


### QueryEngineTool

In [46]:
# 7.2
from llama_index.core.tools import QueryEngineTool
# 7.0 Query Data
vector_query_engine = index.as_query_engine()

desc = "Your job is to query the stored data from file data.csv but NOT to search the "
read_tool = QueryEngineTool.from_defaults(
                                             query_engine=vector_query_engine,
                                             description=( desc
                                                           
                                                         ),
                                            )

In [47]:
from tavily import AsyncTavilyClient

async def search_jobs_on_web(query: str) -> str:
    """Given the degree and specialization, search for jobs on the web where he can apply as per his qualifications. You are not to search for any other information on the web."""
    client = AsyncTavilyClient(api_key="apikey")
    return str(await client.search(query))

In [48]:

async def course_recommender(query: str) -> str:
    """Given the job requirments, recommend courses from the web ."""
    client = AsyncTavilyClient(api_key="apikey")
    return str(await client.search(query))

In [49]:
from llama_index.core.tools import FunctionTool

search_web_tool = FunctionTool.from_defaults(fn= search_jobs_on_web)
course_recommender_tool = FunctionTool.from_defaults(fn=course_recommender)

In [50]:
# 9.0

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

# 9.1 Define workers
agent_worker = FunctionCallingAgentWorker.from_tools(
                                                      [read_tool, search_web_tool, course_recommender_tool], 
                                                      llm=llm, 
                                                      verbose= True,  # Try also False
                                                    )

# 9.2 Define supervisor
agent = AgentRunner(agent_worker)

## Chatting

In [52]:

response = agent.chat(
                      "Give all details of 'Rahul Kumar' having age of 23 from the data in the database"
                      )

print(response)

Added user message to memory: Give all details of 'Rahul Kumar' having age of 23 from the data in the database
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "Rahul Kumar age 23"}
=== Function Output ===
Rahul Kumar was born on 3/26/2002 and has a CGPA of 7.7 in MBA. He completed his B.Com. (Hons.) in Finance from St. Xavier's College, Amity University in 2022 with 74.16%. He attended Delhi Public School and scored 85.33% in class 12 from the Commerce stream under the CBSE board.
=== LLM Response ===
If you have any more questions or need further assistance, feel free to ask!


In [56]:
response = agent.chat("Search the web to list some jobs for him")
print(response)

Added user message to memory: Search the web to list some jobs for him
=== Calling Function ===
Calling function: search_jobs_on_web with args: {"query": "MBA jobs"}
=== Function Output ===
{'query': 'MBA jobs', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': '20 High-Paying Careers With an MBA (With Duties and Salary)', 'url': 'https://in.indeed.com/career-advice/finding-a-job/careers-with-an-mba', 'content': 'Learn about the types of jobs you can get after an MBA degree, along with their duties and salaries. Find out how an MBA can help you advance your career in various fields, such as information technology, healthcare, business operations and more.', 'score': 0.78413767, 'raw_content': None}, {'title': 'MBA Jobs, Employment - Indeed', 'url': 'https://www.indeed.com/q-MBA-jobs.html', 'content': 'MBA jobs. Sort by: relevance - date. 42,000+ jobs. Audit and Reimbursement III. New. Elevance Health 3.6. Indianapolis, IN. Pay information not provided. Fu

In [58]:
response = agent.chat("Recommend some courses for Rahul as he wants to be data analyst")
print(response)

Added user message to memory: Recommend some courses for Rahul as he wants to be data analyst
=== Calling Function ===
Calling function: course_recommender with args: {"query": "data analyst courses"}
=== Function Output ===
{'query': 'data analyst courses', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Best Data Analyst Courses & Certificates [2025] | Coursera Learn Online', 'url': 'https://www.coursera.org/courses?query=data+analyst', 'content': 'Best Data Analyst Courses Online with Certificates [2024] | Coursera Data Analyst Courses Online Explore the Data Analyst Course Catalog ibm data analyst professional certificate In summary, here are 10 of our most popular data analyst courses How can taking online courses help me learn to be a data analyst?\u200e What are the benefits of taking an online Data Analyst course?\u200e Online Data Analyst courses offer a convenient and flexible way to enhance your knowledge or learn new Data Analyst skills. Ch

## Loop to constantly chat

In [ ]:
# You can also set up a loop to constantly query:

while True:
    text_input = input("User: ")
    if text_input == "exit":
        break
    response = agent.chat(text_input)
    print(f"Agent: {response}")

### ============= DONE ===============================

In [ ]:
##################